# <center>LFAToolkit.jl</center>

### <center>Jeremy L Thompson</center>

Local Fourier Analysis Toolkit for analysis and tuning of multigrid and smoothing techniques for finite element operators with arbitrary, user provided weak forms of partial differential equations

<br>
<center>
    <a href="
         https://www.github.com/jeremylt/LFAToolkit.jl">github.com/jeremylt/LFAToolkit.jl</a> 
</center>
<style>
//change background settings {}
div.slide-background {
	border-top: 30px midnightblue solid;
	border-bottom: 30px midnightblue solid;
}
</style>

## <center>Overview</center>

* Introduction
* Mathematical Background
 - Operator Representation
 - Local Fourier Analysis
* PDE Examples
 - Poisson Problem
 - Neo-Hookean Hyperelasticity
* Smoother Examples
 - Jacobi
 - Chebyshev
* Multigrid Examples
 - p-multigrid
 - h-multigrid
* References

## <center>Introduction</center>

Local Fourier Analysis (LFA) of multigrid and preconditioning methods
* models error reduction across frequencies
* predicts convergence of methods
* facilitates parameter tuning

The technique was first introduced by Brandt in 1977<sup>[2]</sup> for h-multigrid with finite differences

Technique was later adapted for finite elements and additional preconditioning techniques

## <center>Mathematical Background</center>

Mathematical representation of finite element operators and Local Fourier Analysis in LFAToolkit.jl

## <center>Mathematical Background</center>
### <center>Operator Representation</center>

LFAToolkit uses a finite element operator representation designed to facilitate matrix-free implementations <sup>[3]</sup>

This representation supports arbitrary second order PDEs

<br>

$$
\int_{\Omega} v \cdot f_0 \left( u, \nabla u \right) + \nabla v : f_1 \left( u, \nabla u \right) = \int_{\Omega} f v, \forall v \in V
$$

for some suitable $V \subseteq H_0^1 \left( \Omega \right)$

where

* $\cdot$ represents contraction over fields
* $:$ represents contraction over fields and spatial dimensions

## <center>Mathematical Background</center>
### <center>Operator Representation</center>

This weak form can be discretized by selecting a finite element basis

<br>

$$
\color{maroon}{\mathbf{A}} u = \sum_e \mathcal{E}^T_e \left[
  \left( \color{royalblue}{\mathbf{B}}_0^e \right)^T \color{royalblue}{\mathbf{W}}^e \Lambda \left( \color{yellowgreen}{\mathbf{f_0}} \left( u^e, \nabla u^e \right) \right)
  + \sum_{i = 1}^d \left( \color{royalblue}{\mathbf{B}}_i^e \right)^T \color{royalblue}{\mathbf{W}}^e \Lambda \left( \color{yellowgreen}{\mathbf{f_1}} \left( u^e, \nabla u^e \right) \right)
\right]
$$

where

<br>

<p style="margin-left:10%;">$u^e = \color{royalblue}{\mathbf{B}}_0^e \mathcal{E}^e u$ and $\nabla u^e = \lbrace \color{royalblue}{\mathbf{B}}_i^e \mathcal{E}^e u \rbrace_{i = 1}^d$</p>

## <center>Mathematical Background</center>
### <center>Operator Representation</center>

This operator can be represented as

<br>

$$
\color{maroon}{\mathbf{A}} = \mathbf{G}^T \color{royalblue}{\mathbf{B}}^T \color{yellowgreen}{\mathbf{D}} \color{royalblue}{\mathbf{B}} \mathbf{G}
$$

where

* $\color{maroon}{\mathbf{A}}$ - full finite element operator
* $\mathbf{G}$ - element restriction or assembly operator
* $\color{royalblue}{\mathbf{B}}$ - element basis operator (providing $\color{royalblue}{\mathbf{B}}_i$)
* $\color{yellowgreen}{\mathbf{D}}$ - diagonal pointwise weak form operator (from $\color{yellowgreen}{\mathbf{f_0}}$, $\color{yellowgreen}{\mathbf{f_1}}$, and element geometry)

## <center>Mathematical Background</center>
### <center>Operator Representation</center>

This representation facilitates matrix-free implementation, such as in libCEED<sup>[1]</sup>
<center>
<a href="https://github.com/CEED/libCEED">github.com/CEED/libCEED</a>
<img src="https://raw.githubusercontent.com/CEED/libCEED/main/doc/img/libCEED.png">
</center>

## <center>Mathematical Background</center>
### <center>Local Fourier Analysis</center>

LFA investigates the local properties of the descretized operator

Consider a scalar Toeplitz operator $L_h$ on the infinite 1D grid $G_h$

<br>

$$
L_h \mathrel{\hat{=}} \left[ s_\kappa \right]_h \left( \kappa \in V \right)\\
L_h w_h \left( x \right) = \sum_{\kappa \in V} s_\kappa w_h \left( x + \kappa h \right)
$$

where

* $V \subset \mathcal{Z}$ is an index set
* $s_\kappa \in \mathcal{R}$ are constant coefficients
* $w_h \left( x \right)$ is a $l^2$ function on $G_h$

## <center>Mathematical Background</center>
### <center>Local Fourier Analysis</center>

If for all $\varphi \left( \theta, x \right)$

<br>

$$
L_h \varphi \left( \theta, x \right) = \tilde{L}_h \left( \theta \right) \varphi \left( \theta, x \right)
$$

then $\tilde{L}_h \left( \theta \right) = \sum_{\kappa \in V} s_\kappa e^{\imath \theta \kappa}$ is the **symbol** of $L_h$

## <center>Mathematical Background</center>
### <center>Local Fourier Analysis</center>

For a system of equations, such as the $p \times p$ linear system of equations for a scalar problem on a single $p$ order finite element

<br>

$$
\tilde{L}_h =
\begin{bmatrix}
    \tilde{L}_h^{1, 1}  &&  \cdots  &&  \tilde{L}_h^{1, p}  \\
    \vdots              &&  \vdots  &&  \vdots              \\
    \tilde{L}_h^{p, 1}  &&  \cdots  &&  \tilde{L}_h^{p, p}  \\
\end{bmatrix}
$$

Multi-component equations are represented in a similar fashion

## <center>Mathematical Background</center>
### <center>Operator Representation + Local Fourier Analysis</center>

We combine these two formulations with a Fourier mode localization operator in 1D

<br>

$$
\tilde{\color{maroon}{\mathbf{A}}} \left( \mathbf{\theta} \right) = \mathbf{Q}^T \left( \color{maroon}{\mathbf{A}}_e \odot \left[ e^{\imath \left( x_i - x_j \right) \theta / h} \right] \right) \mathbf{Q}
$$

where

* $\color{maroon}{\mathbf{A}}_e = \color{royalblue}{\mathbf{B}}^T \color{yellowgreen}{\mathbf{D}} \color{royalblue}{\mathbf{B}}$
* $\mathbf{Q} = 
\begin{bmatrix}
    I    \\
    e_0  \\
\end{bmatrix} =
\begin{bmatrix}
    1       &&  0       &&  \cdots  &&  0       \\
    0       &&  1       &&  \cdots  &&  0       \\
    \vdots  &&  \vdots  &&  \vdots  &&  \vdots  \\
    0       &&  0       &&  \cdots  &&  1       \\
    1       &&  0       &&  \cdots  &&  0       \\
\end{bmatrix}
$

Higher dimensions use with a Kronecker product of the 1D localization operator

## <center>PDE Examples</center>

PDEs can be represented in LFAToolkit.jl by providing bases, a mesh, and a pointwise representation of the weak form

## <center>PDE Example</center>
### <center>Poisson Problem</center>

As the classic example, let's look at the scalar Poisson problem

<br>

$$
\int \nabla v \nabla u = \int f v
$$

$\color{royalblue}{\mathbf{B}}$ represents the finite element basis, such as a H1 Lagrange basis on Gauss-Lobatto points

$\color{yellowgreen}{\mathbf{D}}$ represents the pointwise application of the weak form

In [ ]:
# setup
using LFAToolkit

# mesh
dim = 1
mesh = Mesh1D(1.0)
# basis
p = 2
ncomp = 1
basis = TensorH1LagrangeBasis(p+1, p+2, ncomp, dim)
# weak form
function diffusionweakform(du::Array{Float64}, w::Array{Float64})
    return dv = du*w[1]
end

# operator
inputs = [
    OperatorField(basis, [EvaluationMode.gradient]),
    OperatorField(basis, [EvaluationMode.quadratureweights]),
]
outputs = [OperatorField(basis, [EvaluationMode.gradient])]
A = Operator(diffusionweakform, mesh, inputs, outputs)

# compute symbols
θ = [π]
symbol = computesymbols(A, θ)

In [ ]:
using LFAToolkit
using LinearAlgebra
using Plots

# setup
p = 3
dimension = 1

mesh = []
if dimension == 1
   mesh = Mesh1D(1.0)
elseif dimension == 2
   mesh = Mesh2D(1.0, 1.0)
end

# operator
diffusion = GalleryOperator("diffusion", p, p, mesh)

# compute full operator symbols
numberruns = 250
maxeigenvalue = 0
θ_min = -π/2
θ_max = 3π/2

# compute and plot smoothing factor
# -- 1D --
if dimension == 1
    # compute
    maxeigenvalues = zeros(numberruns)

    for i in 1:numberruns
        θ = [θ_min + (θ_max - θ_min)*i/numberruns]
        A = computesymbols(diffusion, θ)
        eigenvalues = [abs(val) for val in eigvals(A)]
        maxeigenvalues[i] = max(eigenvalues...)
        maxeigenvalue = max(maxeigenvalue, maxeigenvalues[i])
    end

    # plot
    #println("max eigenvalue: ", maxeigenvalue)
    xrange = θ_min/π:(θ_max - θ_min)/π/(numberruns-1):θ_max/π
    plot(
        xrange,
        xlabel="θ/π",
        xtickfont=font(12, "Courier"),
        maxeigenvalues,
        ytickfont=font(12, "Courier"),
        ylabel="spectral radius",
        linewidth=3,
        legend=:none,
        title="Diffusion Operator Symbol"
    )
    ylims!(0.0, max(maxeigenvalues...) * 1.1)
# -- 2D --
elseif dimension == 2
    # compute
    maxeigenvalues = zeros(numberruns, numberruns)

    for i in 1:numberruns, j in 1:numberruns
        θ = [
            θ_min + (θ_max - θ_min)*i/numberruns,
            θ_min + (θ_max - θ_min)*j/numberruns
        ]
        A = computesymbols(diffusion, θ)
        eigenvalues = [abs(val) for val in eigvals(A)]
        maxeigenvalues[i, j] = max(eigenvalues...)
        maxeigenvalue = max(maxeigenvalue, maxeigenvalues[i, j])
    end

    # plot
    #println("max eigenvalue: ", maxeigenvalue)
    xrange = θ_min/π:(θ_max - θ_min)/π/(numberruns-1):θ_max/π
    heatmap(
        xrange,
        xlabel="θ/π",
        xtickfont=font(12, "Courier"),
        xrange,
        ylabel="θ/π",
        maxeigenvalues,
        ytickfont=font(12, "Courier"),
        title="Diffusion Operator Symbol",
        transpose=true,
        aspect_ratio=:equal
    )
    xlims!(θ_min/π, θ_max/π)
    ylims!(θ_min/π, θ_max/π)
end

In [ ]:
using LFAToolkit
using LinearAlgebra
using Plots

# setup
p = 3
dimension = 2

mesh = []
if dimension == 1
   mesh = Mesh1D(1.0)
elseif dimension == 2
   mesh = Mesh2D(1.0, 1.0)
end

# operator
diffusion = GalleryOperator("diffusion", p, p, mesh)

# compute full operator symbols
numberruns = 250
maxeigenvalue = 0
θ_min = -π/2
θ_max = 3π/2

# compute and plot smoothing factor
# -- 1D --
if dimension == 1
    # compute
    maxeigenvalues = zeros(numberruns)

    for i in 1:numberruns
        θ = [θ_min + (θ_max - θ_min)*i/numberruns]
        A = computesymbols(diffusion, θ)
        eigenvalues = [abs(val) for val in eigvals(A)]
        maxeigenvalues[i] = max(eigenvalues...)
        maxeigenvalue = max(maxeigenvalue, maxeigenvalues[i])
    end

    # plot
    #println("max eigenvalue: ", maxeigenvalue)
    xrange = θ_min/π:(θ_max - θ_min)/π/(numberruns-1):θ_max/π
    plot(
        xrange,
        xlabel="θ/π",
        xtickfont=font(12, "Courier"),
        maxeigenvalues,
        ytickfont=font(12, "Courier"),
        ylabel="spectral radius",
        linewidth=3,
        legend=:none,
        title="Diffusion Operator Symbol"
    )
    ylims!(0.0, max(maxeigenvalues...) * 1.1)
# -- 2D --
elseif dimension == 2
    # compute
    maxeigenvalues = zeros(numberruns, numberruns)

    for i in 1:numberruns, j in 1:numberruns
        θ = [
            θ_min + (θ_max - θ_min)*i/numberruns,
            θ_min + (θ_max - θ_min)*j/numberruns
        ]
        A = computesymbols(diffusion, θ)
        eigenvalues = [abs(val) for val in eigvals(A)]
        maxeigenvalues[i, j] = max(eigenvalues...)
        maxeigenvalue = max(maxeigenvalue, maxeigenvalues[i, j])
    end

    # plot
    #println("max eigenvalue: ", maxeigenvalue)
    xrange = θ_min/π:(θ_max - θ_min)/π/(numberruns-1):θ_max/π
    heatmap(
        xrange,
        xlabel="θ/π",
        xtickfont=font(12, "Courier"),
        xrange,
        ylabel="θ/π",
        maxeigenvalues,
        ytickfont=font(12, "Courier"),
        title="Diffusion Operator Symbol",
        transpose=true,
        aspect_ratio=:equal
    )
    xlims!(θ_min/π, θ_max/π)
    ylims!(θ_min/π, θ_max/π)
end

## <center>PDE Example</center>
### <center>Neo-Hookean Hyperelasticity</center>

The strong form of the static balance of linear-momentum at finite strain is given by

<br>

$$
-\nabla_x \cdot \mathbf{P} - \rho_0 \mathbf{g} = \mathbf{0}
$$

where $-\nabla_k$ is the gradient with respect to the reference configuration, $\mathbf{P}$ is the first Piola-Kirchhoff stress tensor, $\rho_0$ is the reference mass density, and $\mathbf{g}$ is the forcing function

The first Piola-Kirchhoff stress tensor is given by

<br>

$$
\mathbf{P} = \mathbf{F} \mathbf{S}
$$

where $\mathbf{F}$ is the deformation gradient and $\mathbf{S}$ is the second Piola-Kirchhoff stress tensor

In [ ]:
# setup
using LFAToolkit
using LinearAlgebra

# fem parameters
dim = 3
mesh = Mesh3D(1.0, 1.0, 1.0)
p = 2
ncomp = 3
basis = TensorH1LagrangeBasis(p+1, p+2, ncomp, dim)

# constants
e = 1E6                     # Young's modulus
ν = 0.3                     # Poisson's ratio
K = e/(3*(1 - 2*ν))         # bulk modulus
λ = e*ν/((1 + ν)*(1 - 2*ν)) # Lamé parameters
μ = e/(2*(1 + ν))

# state vector
gradu = [1; 2; 3]*ones(1, 3);

In [ ]:
# Neo-Hookean weak form
function neohookeanweakform(deltadu::Array{Float64}, w::Array{Float64})
    # dP = dF S + F dS

    F = gradu + I
    J = det(F)
    # Green-Lagrange strain tensor
    E = (gradu*gradu' + gradu'*gradu)/2
    # right Cauchy-Green tensor
    C = 2*E + I
    C_inv = C^-1
    # second Piola-Kirchhoff
    S = λ*log(J)*C_inv + 2*μ*C_inv*E

    deltadu = deltadu'
    dF = deltadu + I
    deltaE = (deltadu*deltadu' + deltadu'*deltadu)/2
    dS = λ*sum(C_inv.*deltaE)*C_inv + 2*(μ - λ*log(J))*C_inv*deltaE*C_inv
    dP = dF*S + F*dS

    return [dP']
end

In [ ]:
# linearized Neo-Hookean operator
inputs = [
    OperatorField(
        basis,
        [EvaluationMode.gradient],
        "gradient of deformation"
        ),
    OperatorField(
        basis,
        [EvaluationMode.quadratureweights],
        ),
]
outputs = [
    OperatorField(
        basis,
        [EvaluationMode.gradient],
        "test function gradient of deformation",
    ),
]
A = Operator(neohookeanweakform, mesh, inputs, outputs)

## <center>Smoother Examples</center>

The error propagation operator for smoothers is given by

<br>

$$
\mathbf{S} = \mathbf{I} - \mathbf{M}^{-1} \color{maroon}{\mathbf{A}}
$$

where $\mathbf{M}^{-1}$ is the smoothing operator

## <center>Smoother Example</center>
### <center>Jacobi</center>

Lets consider simple Jacobi smoothing

<br>

$$
\mathbf{M}^{-1} = \omega \space \text{diag} \left( \color{maroon}{\mathbf{A}} \right)^{-1}
$$

The error propagation **symbol** is given by

<br>

$$
\tilde{\mathbf{S}} \left( \omega, \mathbf{\theta} \right) = \mathbf{I} - \omega \space \left( \mathbf{Q}^T \text{diag} \left( \color{maroon}{\mathbf{A}}_e \right) \mathbf{Q} \right) \tilde{\color{maroon}{\mathbf{A}}} \left( \mathbf{\theta} \right)
$$

In [ ]:
# setup
using LFAToolkit

# mesh
dim = 2
mesh = Mesh2D(1.0, 1.0)

# operator
A = GalleryOperator("diffusion", p+1, p+2, mesh)
S = Jacobi(A)

# compute symbols
ω = [1.0] # smoothing weight
θ = [π, π]
symbol = computesymbols(S, ω, θ)

In [ ]:
using LFAToolkit
using LinearAlgebra
using Plots

# setup
p = 3
dimension = 1

mesh = []
if dimension == 1
   mesh = Mesh1D(1.0)
elseif dimension == 2
   mesh = Mesh2D(1.0, 1.0)
end

# operator
diffusion = GalleryOperator("diffusion", p, p, mesh)

# Jacobi smoother
jacobi = Jacobi(diffusion)

# full operator symbols
numberruns = 250
maxeigenvalue = 0
θ_min = -π/2
θ_max = 3π/2

# compute and plot smoothing factor
# -- 1D --
if dimension == 1
    # setup
    ω = [0.77]
    maxeigenvalues = zeros(numberruns)

    # compute
    for i in 1:numberruns
        θ = [θ_min + (θ_max - θ_min)*i/numberruns]
        if abs(θ[1] % 2π) >  π/256
            A = computesymbols(jacobi, ω, θ)
            eigenvalues = [abs(val) for val in eigvals(A)]
            maxeigenvalues[i] = max(eigenvalues...)
            maxeigenvalue = max(maxeigenvalue, maxeigenvalues[i])
        end
    end

    # plot
    #println("max eigenvalue: ", maxeigenvalue)
    xrange = θ_min/π:(θ_max - θ_min)/π/(numberruns-1):θ_max/π
    plot(
        xrange,
        xlabel="θ/π",
        xtickfont=font(12, "Courier"),
        maxeigenvalues,
        ytickfont=font(12, "Courier"),
        ylabel="spectral radius",
        linewidth=3,
        legend=:none,
        title="Jacobi Operator Symbol"
    )
    ylims!(0.0, max(maxeigenvalues...) * 1.1)
# -- 2D --
elseif dimension == 2
    # setup
    ω = [1.0]
    maxeigenvalues = zeros(numberruns, numberruns)

    # compute
    for i in 1:numberruns, j in 1:numberruns
        θ = [
            θ_min + (θ_max - θ_min)*i/numberruns,
            θ_min + (θ_max - θ_min)*j/numberruns
        ]
        if sqrt(abs(θ[1] % 2π)^2 + abs(θ[2] % 2π)^2) > π/128
            A = computesymbols(jacobi, ω, θ)
            eigenvalues = [abs(val) for val in eigvals(A)]
            maxeigenvalues[i, j] = max(eigenvalues...)
            maxeigenvalue = max(maxeigenvalue, maxeigenvalues[i, j])
        end
    end

    # plot
    #println("max eigenvalue: ", maxeigenvalue)
    xrange = θ_min/π:(θ_max - θ_min)/π/(numberruns-1):θ_max/π
    heatmap(
        xrange,
        xlabel="θ/π",
        xtickfont=font(12, "Courier"),
        xrange,
        ylabel="θ/π",
        ytickfont=font(12, "Courier"),
        maxeigenvalues,
        title="Jacobi Operator Symbol",
        transpose=true,
        aspect_ratio=:equal
    )
    xlims!(θ_min/π, θ_max/π)
    ylims!(θ_min/π, θ_max/π)
end

In [ ]:
using LFAToolkit
using LinearAlgebra
using Plots

# setup
p = 3
dimension = 2

mesh = []
if dimension == 1
   mesh = Mesh1D(1.0)
elseif dimension == 2
   mesh = Mesh2D(1.0, 1.0)
end

# operator
diffusion = GalleryOperator("diffusion", p, p, mesh)

# Jacobi smoother
jacobi = Jacobi(diffusion)

# full operator symbols
numberruns = 250
maxeigenvalue = 0
θ_min = -π/2
θ_max = 3π/2

# compute and plot smoothing factor
# -- 1D --
if dimension == 1
    # setup
    ω = [1.0]
    maxeigenvalues = zeros(numberruns)

    # compute
    for i in 1:numberruns
        θ = [θ_min + (θ_max - θ_min)*i/numberruns]
        if abs(θ[1] % 2π) >  π/128
            A = computesymbols(jacobi, ω, θ)
            eigenvalues = [abs(val) for val in eigvals(A)]
            maxeigenvalues[i] = max(eigenvalues...)
            maxeigenvalue = max(maxeigenvalue, maxeigenvalues[i])
        end
    end

    # plot
    #println("max eigenvalue: ", maxeigenvalue)
    xrange = θ_min/π:(θ_max - θ_min)/π/(numberruns-1):θ_max/π
    plot(
        xrange,
        xlabel="θ/π",
        xtickfont=font(12, "Courier"),
        maxeigenvalues,
        ytickfont=font(12, "Courier"),
        ylabel="spectral radius",
        linewidth=3,
        legend=:none,
        title="Jacobi Operator Symbol"
    )
    ylims!(0.0, max(maxeigenvalues...) * 1.1)
# -- 2D --
elseif dimension == 2
    # setup
    ω = [1.0]
    maxeigenvalues = zeros(numberruns, numberruns)

    # compute
    for i in 1:numberruns, j in 1:numberruns
        θ = [
            θ_min + (θ_max - θ_min)*i/numberruns,
            θ_min + (θ_max - θ_min)*j/numberruns
        ]
        if sqrt(abs(θ[1] % 2π)^2 + abs(θ[2] % 2π)^2) > π/128
            A = computesymbols(jacobi, ω, θ)
            eigenvalues = [abs(val) for val in eigvals(A)]
            maxeigenvalues[i, j] = max(eigenvalues...)
            maxeigenvalue = max(maxeigenvalue, maxeigenvalues[i, j])
        end
    end

    # plot
    #println("max eigenvalue: ", maxeigenvalue)
    xrange = θ_min/π:(θ_max - θ_min)/π/(numberruns-1):θ_max/π
    heatmap(
        xrange,
        xlabel="θ/π",
        xtickfont=font(12, "Courier"),
        xrange,
        ylabel="θ/π",
        ytickfont=font(12, "Courier"),
        maxeigenvalues,
        title="Jacobi Operator Symbol",
        transpose=true,
        aspect_ratio=:equal
    )
    xlims!(θ_min/π, θ_max/π)
    ylims!(θ_min/π, θ_max/π)
end

## <center>Smoother Example</center>
### <center>Chebyshev</center>

Lets consider Chebyshev polynomial smoothing

<br>

$$
\mathbf{M}^{-1} = \rho_q \left( \text{diag} \left( \color{maroon}{\mathbf{A}} \right)^{-1} \color{maroon}{\mathbf{A}} \right)
$$

where

$\rho$ is derived from the Chebyshev polynomials<sup>[4]</sup> and $q$ is the polynomial order of the smoother

The error propagation **symbol** is given by

<br>

$$
\tilde{\mathbf{S}} \left( q, \mathbf{\theta} \right) = \mathbf{I} - \rho_q \left( \left( \mathbf{Q}^T \text{diag} \left( \color{maroon}{\mathbf{A}}_e \right) \mathbf{Q} \right)^{-1} \tilde{\color{maroon}{\mathbf{A}}} \left( \mathbf{\theta} \right) \right) \tilde{\color{maroon}{\mathbf{A}}} \left( \mathbf{\theta} \right)
$$

In [ ]:
# setup
using LFAToolkit

# mesh
dim = 2
mesh = Mesh2D(1.0, 1.0)

# operator
A = GalleryOperator("diffusion", p+1, p+2, mesh)
S = Chebyshev(A)

# compute symbols
ρ = [3] # Chebyshev order
θ = [π, π]
symbol = computesymbols(S, ρ, θ)

In [ ]:
using LFAToolkit
using LinearAlgebra
using Plots

# setup
p = 3
dimension = 1

mesh = []
if dimension == 1
   mesh = Mesh1D(1.0)
elseif dimension == 2
   mesh = Mesh2D(1.0, 1.0)
end

# operator
diffusion = GalleryOperator("diffusion", p, p, mesh)

# Chebyshev smoother
chebyshev = Chebyshev(diffusion)

# full operator symbols
numberruns = 250
maxeigenvalue = 0
θ_min = -π/2
θ_max = 3π/2

# compute and plot smoothing factor
# -- 1D --
if dimension == 1
    # setup
    ω = [3]
    maxeigenvalues = zeros(numberruns)

    # compute
    for i in 1:numberruns
        θ = [θ_min + (θ_max - θ_min)*i/numberruns]
        if abs(θ[1] % 2π) >  π/256
            A = computesymbols(chebyshev, ω, θ)
            eigenvalues = [abs(val) for val in eigvals(A)]
            maxeigenvalues[i] = max(eigenvalues...)
            maxeigenvalue = max(maxeigenvalue, maxeigenvalues[i])
        end
    end

    # plot
    #println("max eigenvalue: ", maxeigenvalue)
    xrange = θ_min/π:(θ_max - θ_min)/π/(numberruns-1):θ_max/π
    plot(
        xrange,
        xlabel="θ/π",
        xtickfont=font(12, "Courier"),
        maxeigenvalues,
        ytickfont=font(12, "Courier"),
        ylabel="spectral radius",
        linewidth=3,
        legend=:none,
        title="Chebyshev Operator Symbol"
    )
    ylims!(0.0, max(maxeigenvalues...) * 1.1)
# -- 2D --
elseif dimension == 2
    # setup
    ω = [3]
    maxeigenvalues = zeros(numberruns, numberruns)

    # compute
    for i in 1:numberruns, j in 1:numberruns
        θ = [
            θ_min + (θ_max - θ_min)*i/numberruns,
            θ_min + (θ_max - θ_min)*j/numberruns
        ]
        if sqrt(abs(θ[1] % 2π)^2 + abs(θ[2] % 2π)^2) > π/128
            A = computesymbols(chebyshev, ω, θ)
            eigenvalues = [abs(val) for val in eigvals(A)]
            maxeigenvalues[i, j] = max(eigenvalues...)
            maxeigenvalue = max(maxeigenvalue, maxeigenvalues[i, j])
        end
    end

    # plot
    #println("max eigenvalue: ", maxeigenvalue)
    xrange = θ_min/π:(θ_max - θ_min)/π/(numberruns-1):θ_max/π
    heatmap(
        xrange,
        xlabel="θ/π",
        xtickfont=font(12, "Courier"),
        xrange,
        ylabel="θ/π",
        ytickfont=font(12, "Courier"),
        maxeigenvalues,
        title="Chebyshev Operator Symbol",
        transpose=true,
        aspect_ratio=:equal
    )
    xlims!(θ_min/π, θ_max/π)
    ylims!(θ_min/π, θ_max/π)
end

In [ ]:
using LFAToolkit
using LinearAlgebra
using Plots

# setup
p = 3
dimension = 2

mesh = []
if dimension == 1
   mesh = Mesh1D(1.0)
elseif dimension == 2
   mesh = Mesh2D(1.0, 1.0)
end

# operator
diffusion = GalleryOperator("diffusion", p, p, mesh)

# Chebyshev smoother
chebyshev = Chebyshev(diffusion)

# full operator symbols
numberruns = 250
maxeigenvalue = 0
θ_min = -π/2
θ_max = 3π/2

# compute and plot smoothing factor
# -- 1D --
if dimension == 1
    # setup
    ω = [3]
    maxeigenvalues = zeros(numberruns)

    # compute
    for i in 1:numberruns
        θ = [θ_min + (θ_max - θ_min)*i/numberruns]
        if abs(θ[1] % 2π) >  π/128
            A = computesymbols(chebyshev, ω, θ)
            eigenvalues = [abs(val) for val in eigvals(A)]
            maxeigenvalues[i] = max(eigenvalues...)
            maxeigenvalue = max(maxeigenvalue, maxeigenvalues[i])
        end
    end

    # plot
    #println("max eigenvalue: ", maxeigenvalue)
    xrange = θ_min/π:(θ_max - θ_min)/π/(numberruns-1):θ_max/π
    plot(
        xrange,
        xlabel="θ/π",
        xtickfont=font(12, "Courier"),
        maxeigenvalues,
        ytickfont=font(12, "Courier"),
        ylabel="spectral radius",
        linewidth=3,
        legend=:none,
        title="Chebyshev Operator Symbol"
    )
    ylims!(0.0, max(maxeigenvalues...) * 1.1)
# -- 2D --
elseif dimension == 2
    # setup
    ω = [3]
    maxeigenvalues = zeros(numberruns, numberruns)

    # compute
    for i in 1:numberruns, j in 1:numberruns
        θ = [
            θ_min + (θ_max - θ_min)*i/numberruns,
            θ_min + (θ_max - θ_min)*j/numberruns
        ]
        if sqrt(abs(θ[1] % 2π)^2 + abs(θ[2] % 2π)^2) > π/128
            A = computesymbols(chebyshev, ω, θ)
            eigenvalues = [abs(val) for val in eigvals(A)]
            maxeigenvalues[i, j] = max(eigenvalues...)
            maxeigenvalue = max(maxeigenvalue, maxeigenvalues[i, j])
        end
    end

    # plot
    #println("max eigenvalue: ", maxeigenvalue)
    xrange = θ_min/π:(θ_max - θ_min)/π/(numberruns-1):θ_max/π
    heatmap(
        xrange,
        xlabel="θ/π",
        xtickfont=font(12, "Courier"),
        xrange,
        ylabel="θ/π",
        ytickfont=font(12, "Courier"),
        maxeigenvalues,
        title="Chebyshev Operator Symbol",
        transpose=true,
        aspect_ratio=:equal
    )
    xlims!(θ_min/π, θ_max/π)
    ylims!(θ_min/π, θ_max/π)
end

## <center>Multigrid Examples</center>

The error propagation of a multigrid method is given by

<br>

$$
\mathbf{M}_{TMG} = \mathbf{S}_f \left( \mathbf{I} - \mathbf{P}_{ctof} \color{maroon}{\mathbf{A}}_c^{-1} \mathbf{R}_{ftoc} \color{maroon}{\mathbf{A}}_f \right) \mathbf{S}_f
$$

where the grid transfer operators $\mathbf{P}_{ctof}$ and $\mathbf{R}_{ftoc}$ determine the type of multigrid method

## <center>Multigrid Example</center>
### <center>P-Multigrid</center>

Prolongation given by interpolating low order basis functions to high order basis nodes, restriction by the transpose

<br>

$$
\mathbf{P}_{ctof} = \mathbf{P}_f^T \color{yellowgreen}{\mathbf{D}}_{scale} \color{royalblue}{\mathbf{B}}_{ctof} \mathbf{P}_c, \ \ \mathbf{R}_{ftoc} = \mathbf{P}_{ctof}^T
$$

In [ ]:
using LFAToolkit
using Plots

# Lagrange plotting function
function lagrange(x, i, p)
    points = LFAToolkit.lobattoquadrature(p, false)
    xi = points[i]
    return prod([if (xi == xj) 1 else (x - xj)/(xi - xj) end for xj in points])
end

# coarse element
x = -1:0.05:1
pcoarse = 3
coarse = [[lagrange(xx, i, pcoarse) for xx in x] for i in 1:pcoarse]
fine = [[lagrange(xx, i, 4) + 1.25 for xx in x] for i in 1:4]
plot(
    x,
    coarse,
    yticks = false,
    yshowaxis = false,
    grid = false,
    legend = false,
    color = :blue,
    size = (600, 300),
)
xi = LFAToolkit.lobattoquadrature(pcoarse, false)
yi = zeros(pcoarse)
scatter!(
    xi,
    yi,
    yticks = false,
    yshowaxis = false,
    grid = false,
    legend = false,
    markersize = 5,
    markercolor = :black,
)

# fine element
shift = 1.25
pfine = 5
fine = [[lagrange(xx, i, pfine) + shift for xx in x] for i in 1:pfine]
plot!(
    x,
    fine,
    yticks = false,
    yshowaxis = false,
    grid = false,
    legend = false,
    color = :green
)
xi = LFAToolkit.lobattoquadrature(pfine, false)
yi = ones(pfine)*shift
scatter!(
    xi,
    yi,
    yticks = false,
    yshowaxis = false,
    grid = false,
    legend = false,
    markersize = 5,
    markercolor = :black,
)

In [ ]:
# setup
using LFAToolkit

# mesh
dim = 2
mesh = Mesh2D(1.0, 1.0)

# setup
finep = 2
coarsep = 1
ncomp = 1

# diffusion operators
finediffusion = GalleryOperator("diffusion", finep+1, finep+1, mesh)
coarsediffusion = GalleryOperator("diffusion", coarsep+1, finep+1, mesh)

# prolongation basis
ctofbasis = TensorH1LagrangeBasis(coarsep+1, finep+1, ncomp, dim, lagrangequadrature = true)

# Chebyshev smoother
S = Chebyshev(finediffusion)

In [ ]:
# p-multigrid preconditioner
M = PMultigrid(finediffusion, coarsediffusion, S, [ctofbasis])

# compute symbols
ρ = [3]
v = [1, 1]
θ = [π, π]
symbol = computesymbols(M, ρ, v, θ)

In [ ]:
using LFAToolkit
using LinearAlgebra
using Plots

# setup
finep = 3
coarsep = 2
numbercomponents = 1
dimension = 1

mesh = []
if dimension == 1
   mesh = Mesh1D(1.0)
elseif dimension == 2
   mesh = Mesh2D(1.0, 1.0)
end

ctofbasis = TensorH1LagrangePProlongationBasis(coarsep, finep, numbercomponents, dimension)

# diffusion operators
finediffusion = GalleryOperator("diffusion", finep, finep, mesh)
coarsediffusion = GalleryOperator("diffusion", coarsep, finep, mesh)

# Jacobi smoother
chebyshev = Chebyshev(finediffusion)

# p-multigrid preconditioner
multigrid = PMultigrid(finediffusion, coarsediffusion, chebyshev, [ctofbasis])

# full operator symbols
numberruns = 250
maxeigenvalue = 0
θ_min = -π/2
θ_max = 3π/2

# compute and plot smoothing factor
# -- 1D --
if dimension == 1
    # setup
    ω = [3]
    v = [1, 1]
    maxeigenvalues = zeros(numberruns)

    # compute
    for i in 1:numberruns
        θ = [θ_min + (θ_max - θ_min)*i/numberruns]
        if abs(θ[1]  % 2π) >  π/256
            A = computesymbols(multigrid, ω, v, θ)
            eigenvalues = [abs(val) for val in eigvals(A)]
            maxeigenvalues[i] = max(eigenvalues...)
            maxeigenvalue = max(maxeigenvalue, maxeigenvalues[i])
        end
    end

    # plot
    #println("max eigenvalue: ", maxeigenvalue)
    xrange = θ_min/π:(θ_max - θ_min)/π/(numberruns-1):θ_max/π
    plot(
        xrange,
        xlabel="θ/π",
        xtickfont=font(12, "Courier"),
        maxeigenvalues,
        ytickfont=font(12, "Courier"),
        ylabel="spectral radius",
        linewidth=3,
        legend=:none,
        title="P-Multigrid Error Symbol"
    )
    ylims!(0.0, max(maxeigenvalues...) * 1.1)
# -- 2D --
elseif dimension == 2
    # setup
    ω = [3]
    v = [1, 1]
    maxeigenvalues = zeros(numberruns, numberruns)

    # compute
    for i in 1:numberruns, j in 1:numberruns
        θ = [
            θ_min + (θ_max - θ_min)*i/numberruns,
            θ_min + (θ_max - θ_min)*j/numberruns
        ]
        if sqrt(abs(θ[1] % 2π)^2 + abs(θ[2] % 2π)^2) > π/128
            A = computesymbols(multigrid, ω, v, θ)
            eigenvalues = [abs(val) for val in eigvals(A)]
            maxeigenvalues[i, j] = max(eigenvalues...)
            maxeigenvalue = max(maxeigenvalue, maxeigenvalues[i, j])
        end
    end

    # plot
    #println("max eigenvalue: ", maxeigenvalue)
    xrange = θ_min/π:(θ_max - θ_min)/π/(numberruns-1):θ_max/π
    heatmap(
        xrange,
        xlabel="θ/π",
        xtickfont=font(12, "Courier"),
        xrange,
        ylabel="θ/π",
        ytickfont=font(12, "Courier"),
        maxeigenvalues,
        title="P-Multigrid Error Symbol",
        transpose=true,
        aspect_ratio=:equal
    )
    xlims!(θ_min/π, θ_max/π)
    ylims!(θ_min/π, θ_max/π)
end

In [ ]:
using LFAToolkit
using LinearAlgebra
using Plots

# setup
finep = 3
coarsep = 2
numbercomponents = 1
dimension = 2

mesh = []
if dimension == 1
   mesh = Mesh1D(1.0)
elseif dimension == 2
   mesh = Mesh2D(1.0, 1.0)
end

ctofbasis = TensorH1LagrangePProlongationBasis(coarsep, finep, numbercomponents, dimension)

# diffusion operators
finediffusion = GalleryOperator("diffusion", finep, finep, mesh)
coarsediffusion = GalleryOperator("diffusion", coarsep, finep, mesh)

# Jacobi smoother
chebyshev = Chebyshev(finediffusion)

# p-multigrid preconditioner
multigrid = PMultigrid(finediffusion, coarsediffusion, chebyshev, [ctofbasis])

# full operator symbols
numberruns = 250
maxeigenvalue = 0
θ_min = -π/2
θ_max = 3π/2

# compute and plot smoothing factor
# -- 1D --
if dimension == 1
    # setup
    ω = [3]
    v = [1, 1]
    maxeigenvalues = zeros(numberruns)

    # compute
    for i in 1:numberruns
        θ = [θ_min + (θ_max - θ_min)*i/numberruns]
        if abs(θ[1]  % 2π) >  π/256
            A = computesymbols(multigrid, ω, v, θ)
            eigenvalues = [abs(val) for val in eigvals(A)]
            maxeigenvalues[i] = max(eigenvalues...)
            maxeigenvalue = max(maxeigenvalue, maxeigenvalues[i])
        end
    end

    # plot
    #println("max eigenvalue: ", maxeigenvalue)
    xrange = θ_min/π:(θ_max - θ_min)/π/(numberruns-1):θ_max/π
    plot(
        xrange,
        xlabel="θ/π",
        xtickfont=font(12, "Courier"),
        maxeigenvalues,
        ytickfont=font(12, "Courier"),
        ylabel="spectral radius",
        linewidth=3,
        legend=:none,
        title="P-Multigrid Error Symbol"
    )
    ylims!(0.0, max(maxeigenvalues...) * 1.1)
# -- 2D --
elseif dimension == 2
    # setup
    ω = [3]
    v = [1, 1]
    maxeigenvalues = zeros(numberruns, numberruns)

    # compute
    for i in 1:numberruns, j in 1:numberruns
        θ = [
            θ_min + (θ_max - θ_min)*i/numberruns,
            θ_min + (θ_max - θ_min)*j/numberruns
        ]
        if sqrt(abs(θ[1] % 2π)^2 + abs(θ[2] % 2π)^2) > π/128
            A = computesymbols(multigrid, ω, v, θ)
            eigenvalues = [abs(val) for val in eigvals(A)]
            maxeigenvalues[i, j] = max(eigenvalues...)
            maxeigenvalue = max(maxeigenvalue, maxeigenvalues[i, j])
        end
    end

    # plot
    #println("max eigenvalue: ", maxeigenvalue)
    xrange = θ_min/π:(θ_max - θ_min)/π/(numberruns-1):θ_max/π
    heatmap(
        xrange,
        xlabel="θ/π",
        xtickfont=font(12, "Courier"),
        xrange,
        ylabel="θ/π",
        ytickfont=font(12, "Courier"),
        maxeigenvalues,
        title="P-Multigrid Error Symbol",
        transpose=true,
        aspect_ratio=:equal
    )
    xlims!(θ_min/π, θ_max/π)
    ylims!(θ_min/π, θ_max/π)
end

## <center>Multigrid Example</center>
### <center>H-Multigrid</center>

H-multigrid prolongation is interpolation from a single element to a *macro-element* with multiple sub *micro-elements*

In [ ]:
using LFAToolkit
using Plots

# Lagrange plotting function
function lagrange(x, i, p)
    points = LFAToolkit.lobattoquadrature(p, false)
    xi = points[i]
    return prod([if (xi == xj) 1 else (x - xj)/(xi - xj) end for xj in points])
end

# coarse element
x = -1:0.05:1
pcoarse = 2
coarse = [[lagrange(xx, i, pcoarse) for xx in x] for i in 1:pcoarse]
fine = [[lagrange(xx, i, 4) + 1.25 for xx in x] for i in 1:4]
plot(
    x,
    coarse,
    yticks = false,
    yshowaxis = false,
    grid = false,
    legend = false,
    color = :blue,
    size = (600, 300),
)
xi = LFAToolkit.lobattoquadrature(pcoarse, false)
yi = zeros(pcoarse)
scatter!(
    xi,
    yi,
    yticks = false,
    yshowaxis = false,
    grid = false,
    legend = false,
    markersize = 5,
    markercolor = :black,
)

# left fine element
shift = 1.25
pfine = 2
x = x ./ 2 .- 0.5
fine = [[yi .+ shift] for yi in coarse]
plot!(
    x,
    fine,
    yticks = false,
    yshowaxis = false,
    grid = false,
    legend = false,
    color = :green
)
xi = LFAToolkit.lobattoquadrature(pfine, false) ./ 2 .- 0.5
yi = ones(pfine)*shift
scatter!(
    xi,
    yi,
    yticks = false,
    yshowaxis = false,
    grid = false,
    legend = false,
    markersize = 5,
    markercolor = :black,
)

# right fine element
shift = 1.25
x = x .+ 1
fine = [[yi .+ shift] for yi in coarse]
plot!(
    x,
    fine,
    yticks = false,
    yshowaxis = false,
    grid = false,
    legend = false,
    color = :green
)
xi = LFAToolkit.lobattoquadrature(pfine, false) ./ 2 .+ 0.5
yi = ones(pfine)*shift
scatter!(
    xi,
    yi,
    yticks = false,
    yshowaxis = false,
    grid = false,
    legend = false,
    markersize = 5,
    markercolor = :black,
)

In [ ]:
#setup
p = 1
numbercomponents = 1
dimension = 1

mesh = []
if dimension == 1
   mesh = Mesh1D(1.0)
elseif dimension == 2
   mesh = Mesh2D(1.0, 1.0)
end

numberfineelements1d = 2
ctofbasis = TensorH1LagrangeHProlongationBasis(p+1, numbercomponents, dimension, numberfineelements1d);

# operators
finediffusion = GalleryMacroElementOperator("diffusion", p+1, p+2, numberfineelements1d, mesh);
coarsediffusion = GalleryOperator("diffusion", p+1, p+2, mesh);

# Chebyshev smoother
S = Chebyshev(finediffusion)

In [ ]:
# h-multigrid preconditioner
M = HMultigrid(finediffusion, coarsediffusion, S, [ctofbasis])

# compute symbols
ρ = [3]
v = [1, 1]
θ = [π, π]
symbol = computesymbols(M, ρ, v, θ)

In [ ]:
using LFAToolkit
using LinearAlgebra
using Plots

#setup
p = 2
numbercomponents = 1
dimension = 1

mesh = []
if dimension == 1
   mesh = Mesh1D(1.0)
elseif dimension == 2
   mesh = Mesh2D(1.0, 1.0)
end

numberfineelements1d = 2
ctofbasis = TensorH1LagrangeHProlongationBasis(p, numbercomponents, dimension, numberfineelements1d);

# operators
function diffusionweakform(du::Array{Float64}, w::Array{Float64})
    dv = du*w[1]
    return [dv]
end
# -- fine
basis = TensorH1LagrangeMacroBasis(p, p + 1, numbercomponents, dimension, numberfineelements1d);
inputs = [
    OperatorField(basis, [EvaluationMode.gradient]),
    OperatorField(basis, [EvaluationMode.quadratureweights]),
];
outputs = [OperatorField(basis, [EvaluationMode.gradient])];
finediffusion = Operator(diffusionweakform, mesh, inputs, outputs);
# -- coarse
basis = TensorH1LagrangeBasis(p, p + 1, numbercomponents, dimension);
inputs = [
    OperatorField(basis, [EvaluationMode.gradient]),
    OperatorField(basis, [EvaluationMode.quadratureweights]),
];
outputs = [OperatorField(basis, [EvaluationMode.gradient])];
coarsediffusion = Operator(diffusionweakform, mesh, inputs, outputs);

# Chebyshev smoother
chebyshev = Chebyshev(finediffusion)

# h-multigrid preconditioner
multigrid = HMultigrid(finediffusion, coarsediffusion, chebyshev, [ctofbasis])

# full operator symbols
numberruns = 250
maxeigenvalue = 0
θ_min = -1π/2
θ_max = 3π/2

# compute and plot smoothing factor
# -- 1D --
if dimension == 1
    # setup
    ω = [3]
    v = [1, 1]
    maxeigenvalues = zeros(numberruns)

    # compute
    for i in 1:numberruns
        θ = [θ_min + (θ_max - θ_min)*i/numberruns]
        if abs(θ[1]  % 2π) >  π/256
            A = computesymbols(multigrid, ω, v, θ)
            eigenvalues = [abs(val) for val in eigvals(A)]
            maxeigenvalues[i] = max(eigenvalues...)
            maxeigenvalue = max(maxeigenvalue, maxeigenvalues[i])
        end
    end

    # plot
    #println("max eigenvalue: ", maxeigenvalue)
    xrange = θ_min/π:(θ_max - θ_min)/π/(numberruns-1):θ_max/π
    plot(
        xrange,
        xlabel="θ/π",
        xtickfont=font(12, "Courier"),
        maxeigenvalues,
        ytickfont=font(12, "Courier"),
        ylabel="spectral radius",
        linewidth=3,
        legend=:none,
        title="H-Multigrid Error Symbol"
    )
    ylims!(0.0, max(maxeigenvalues...) * 1.1)
# -- 2D --
elseif dimension == 2
    # setup
    ω = [3]
    v = [1, 1]
    maxeigenvalues = zeros(numberruns, numberruns)

    # compute
    for i in 1:numberruns, j in 1:numberruns
        θ = [
            θ_min + (θ_max - θ_min)*i/numberruns,
            θ_min + (θ_max - θ_min)*j/numberruns
        ]
        if sqrt(abs(θ[1] % 2π)^2 + abs(θ[2] % 2π)^2) > π/128
            A = computesymbols(multigrid, ω, v, θ)
            eigenvalues = [abs(val) for val in eigvals(A)]
            maxeigenvalues[i, j] = max(eigenvalues...)
            maxeigenvalue = max(maxeigenvalue, maxeigenvalues[i, j])
        end
    end

    # plot
    #println("max eigenvalue: ", maxeigenvalue)
    xrange = θ_min/π:(θ_max - θ_min)/π/(numberruns-1):θ_max/π
    heatmap(
        xrange,
        xlabel="θ/π",
        xtickfont=font(12, "Courier"),
        xrange,
        ylabel="θ/π",
        ytickfont=font(12, "Courier"),
        maxeigenvalues,
        title="H-Multigrid Error Symbol",
        transpose=true,
        aspect_ratio=:equal
    )
    xlims!(θ_min/π, θ_max/π)
    ylims!(θ_min/π, θ_max/π)
end

In [ ]:
using LFAToolkit
using LinearAlgebra
using Plots

#setup
p = 2
numbercomponents = 1
dimension = 2

mesh = []
if dimension == 1
   mesh = Mesh1D(1.0)
elseif dimension == 2
   mesh = Mesh2D(1.0, 1.0)
end

numberfineelements1d = 2
ctofbasis = TensorH1LagrangeHProlongationBasis(p, numbercomponents, dimension, numberfineelements1d);

# operators
function diffusionweakform(du::Array{Float64}, w::Array{Float64})
    dv = du*w[1]
    return [dv]
end
# -- fine
basis = TensorH1LagrangeMacroBasis(p, p + 1, numbercomponents, dimension, numberfineelements1d);
inputs = [
    OperatorField(basis, [EvaluationMode.gradient]),
    OperatorField(basis, [EvaluationMode.quadratureweights]),
];
outputs = [OperatorField(basis, [EvaluationMode.gradient])];
finediffusion = Operator(diffusionweakform, mesh, inputs, outputs);
# -- coarse
basis = TensorH1LagrangeBasis(p, p + 1, numbercomponents, dimension);
inputs = [
    OperatorField(basis, [EvaluationMode.gradient]),
    OperatorField(basis, [EvaluationMode.quadratureweights]),
];
outputs = [OperatorField(basis, [EvaluationMode.gradient])];
coarsediffusion = Operator(diffusionweakform, mesh, inputs, outputs);

# Chebyshev smoother
chebyshev = Chebyshev(finediffusion)

# h-multigrid preconditioner
multigrid = HMultigrid(finediffusion, coarsediffusion, chebyshev, [ctofbasis])

# full operator symbols
numberruns = 250
maxeigenvalue = 0
θ_min = -1π/2
θ_max = 3π/2

# compute and plot smoothing factor
# -- 1D --
if dimension == 1
    # setup
    ω = [3]
    v = [1, 1]
    maxeigenvalues = zeros(numberruns)

    # compute
    for i in 1:numberruns
        θ = [θ_min + (θ_max - θ_min)*i/numberruns]
        if abs(θ[1]  % 2π) >  π/256
            A = computesymbols(multigrid, ω, v, θ)
            eigenvalues = [abs(val) for val in eigvals(A)]
            maxeigenvalues[i] = max(eigenvalues...)
            maxeigenvalue = max(maxeigenvalue, maxeigenvalues[i])
        end
    end

    # plot
    #println("max eigenvalue: ", maxeigenvalue)
    xrange = θ_min/π:(θ_max - θ_min)/π/(numberruns-1):θ_max/π
    plot(
        xrange,
        xlabel="θ/π",
        xtickfont=font(12, "Courier"),
        maxeigenvalues,
        ytickfont=font(12, "Courier"),
        ylabel="spectral radius",
        linewidth=3,
        legend=:none,
        title="H-Multigrid Error Symbol"
    )
    ylims!(0.0, max(maxeigenvalues...) * 1.1)
# -- 2D --
elseif dimension == 2
    # setup
    ω = [3]
    v = [1, 1]
    maxeigenvalues = zeros(numberruns, numberruns)

    # compute
    for i in 1:numberruns, j in 1:numberruns
        θ = [
            θ_min + (θ_max - θ_min)*i/numberruns,
            θ_min + (θ_max - θ_min)*j/numberruns
        ]
        if sqrt(abs(θ[1] % 2π)^2 + abs(θ[2] % 2π)^2) > π/128
            A = computesymbols(multigrid, ω, v, θ)
            eigenvalues = [abs(val) for val in eigvals(A)]
            maxeigenvalues[i, j] = max(eigenvalues...)
            maxeigenvalue = max(maxeigenvalue, maxeigenvalues[i, j])
        end
    end

    # plot
    #println("max eigenvalue: ", maxeigenvalue)
    xrange = θ_min/π:(θ_max - θ_min)/π/(numberruns-1):θ_max/π
    heatmap(
        xrange,
        xlabel="θ/π",
        xtickfont=font(12, "Courier"),
        xrange,
        ylabel="θ/π",
        ytickfont=font(12, "Courier"),
        maxeigenvalues,
        title="H-Multigrid Error Symbol",
        transpose=true,
        aspect_ratio=:equal
    )
    xlims!(θ_min/π, θ_max/π)
    ylims!(θ_min/π, θ_max/π)
end

## <center>Questions?</center>

<center>
<a href="https://www.github.com/jeremylt">github.com/jeremylt</a>
<br>
<a href="https://www.github.com/jeremylt/LFAToolkit.jl">github.com/jeremylt/LFAToolkit.jl</a>
</center>

## <center>References</center>

[1] A. Abdelfattah, V. Barra, N. Beams, J. Brown, J.S. Camier, V. Dobrev, Y. Dudouit,L. Ghaffari, T. Kolev, D. Medina, T. Rathnayake, J. L. Thompson,  and S. Tomov, *libCEED User Manual*, Zenodo (2020), https://doi.org/10.5281/zenodo.4302737.

[2] A. Brandt, *Multi-level adaptive solutions to boundary-value problems*, Math. Comp., 31(138) (1977), pp. 33-390.

[3] J. Brown, *Efficient nonlinear solvers for nodal high-order finite elements in 3D*, Journal of Scientific Computing, 45 (2010), pp. 48-63.

[4] M. Gutknecht and S. Röllin,  *The Chebyshev iteration revisited*, Parallel Computing, 28 (2002), pp. 263-283.